<a href="https://colab.research.google.com/github/Iispar/review-summary-API/blob/main/BERT-finetuned-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -q transformers datasets evaluate
!pip install optuna
!pip install accelerate -U
import datasets
import numpy as np
import transformers
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import BertModel
import evaluate
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
model = 'bert-base-cased'
dset = 'mteb/amazon_reviews_multi'

# Preprocessing

The dataset includes reviews from multiple languages so we only import the english ones. The dataset also includes alot of useless data for us, we only need the reviews and their ratings so lets process everything else out.

In [3]:
dataset = datasets.load_dataset(dset, name='en'); # imports the dataset.
# check it works
print(dataset);

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


In [4]:
engDataset = datasets.load_dataset(dset, name='en'); # imports the dataset.
# check it works
print(engDataset);

# FOR TESTING make the dataset smaller
# engDataset["train"] = engDataset["train"].select(range(100000))

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


In [5]:
engDataset = engDataset.shuffle() # shuffle the dataset for safety.
engDataset = engDataset.remove_columns(['id', 'label_text']) # removes everything that we don't need

# Tokenization and padding

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model) # get the basic AutoTokenizer

In [7]:
# tokenizes one example
def tokenize_example(example):
    split = example['text'].split('\n\n'); # splits the sentace and title.
    return tokenizer.encode_plus(split[0], split[1],
             truncation='only_second',
             add_special_tokens=True,
             return_attention_mask=True,
             return_overflowing_tokens=False,
             return_special_tokens_mask=False,
             max_length=512,
             pad_to_max_length=False)


In [8]:
# map the whole dset
eng_tokenized = engDataset.map(tokenize_example)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [9]:
print(eng_tokenized['train'][1])
print(tokenizer.decode(eng_tokenized['train'][1]['input_ids']))

# looks good to me.

{'text': 'Works well when it works\n\nI really like the design of this charger (mostly because I can still use/look at my phone while it\'s charging at night, if I have to. However, there are little things that are very annoying. One is that the blue light that shows you your phone is being charged is quite bright and annoying. I\'ve seen another reviewer say they covered it with tape, but it\'s still a "problem" that people should be aware of. Next, there is an issue with it consistently charging my phone. I have an X in a case and my girlfriend has an 8 in a case (both cases are compatible), but we have to set our phones down on it a certain way or they won\'t charge. Again, this isn\'t a huge deal, but when your product only has one thing it\'s supposed to do and it sometimes doesn\'t even do that, then it\'s worth mentioning.', 'label': 2, 'input_ids': [101, 5853, 1218, 1165, 1122, 1759, 102, 146, 1541, 1176, 1103, 1902, 1104, 1142, 2965, 1197, 113, 2426, 1272, 146, 1169, 1253, 132

# Fine tuning the BERT model for our classification

In [10]:
# config
import torch
import torch.nn as nn

# Create the bert class
class Bert(nn.Module):
    def __init__(self):
        super(Bert, self).__init__()
        # hidden size of BERT (always 768), hidden size of our classifier, and number of labels (in this case 5)
        H_in, H, labels = 768, 25, 5

        # bert is our preloaded distilbert
        self.bert = BertModel.from_pretrained(model)

        # basic one layer feed forward network that outputs the labels.
        self.classifier = nn.Sequential(
            nn.Linear(H_in, H), # bert in.
            nn.ReLU(), # ReLU
            #nn.Dropout(0.5), #dropout if needed.
            nn.Linear(H, labels) # to output labels
        )
    def forward(self, input_ids, attention_mask, labels=None):

        # run the BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        # Extract the last hidden state of the token for classification
        last_hidden_state = outputs[0][:, 0, :]

        # Feed tha last hidden state into the classifier. This outputs the labels.
        logits = self.classifier(last_hidden_state)

        # if there is labels so training
        if labels is not None:
          # calculates the loss.
          loss = torch.nn.CrossEntropyLoss();
          return (loss(logits,labels),logits);
        else:
          # if no labels, just return the logits
          return (logits,);

In [11]:
#calculates the accuracy
accuracy = evaluate.load('accuracy');
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels); # calc accuracy

In [12]:
model = Bert() # init the model

In [13]:
# Training params. We optimize these later.
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = 'steps',
    logging_strategy = 'steps',
    eval_steps = 500,
    logging_steps = 500,
    learning_rate=1.2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    max_steps = 20000,
    num_train_epochs=5,
    weight_decay=0.01,
  )

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # get the data collator with our tokenizer.
early_stopping = transformers.EarlyStoppingCallback(3); # stop training if the eval loss is not getting better.

# Set the trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = eng_tokenized['train'],
    eval_dataset = eng_tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_accuracy,
)

# train the model
trainer.train()

KeyboardInterrupt: ignored

# Hyperparam optimization

This happens over a couple of days so you won't see all the results...

In [ ]:
# Used optuna for optimization

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16])
    epochs=trial.suggest_int('num_train_epochs', low = 2,high = 6),

    # params
    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints",
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        eval_steps = 500,
        logging_steps = 500,
        learning_rate = learning_rate,
        max_steps = 20000,
        load_best_model_at_end = True,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs = epochs
    )

    # the model
    mlp = model
    early_stopping = transformers.EarlyStoppingCallback(3); # stop training if the eval loss is not getting better.

    # train a model
    trainer = transformers.Trainer(
        model = mlp,
        args = trainer_args,
        train_dataset = eng_tokenized['train'],
        eval_dataset = eng_tokenized['test'],
        compute_metrics = compute_accuracy,
        data_collator = data_collator,
        callbacks = [early_stopping]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] # return the best result.

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

[I 2023-10-25 13:05:55,225] A new study created in memory with name: no-name-08a02f1e-c208-4b9b-8932-e22a5b636460
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,1.104300,0.975691,0.579800
1000,0.952600,0.942116,0.591600
1500,0.934100,0.931514,0.606000
2000,0.913600,0.991720,0.587800
2500,0.894900,0.915774,0.600200
3000,0.902400,0.881239,0.617400
3500,0.890700,0.892755,0.615800
4000,0.884800,0.877912,0.622800
4500,0.884100,0.891342,0.612200
5000,0.874100,0.895556,0.623400


[I 2023-10-25 14:23:34,472] Trial 0 finished with value: 0.633 and parameters: {'learning_rate': 7.224710836337679e-05, 'batch_size': 16, 'num_train_epochs': 6}. Best is trial 0 with value: 0.633.


Step,Training Loss,Validation Loss,Accuracy
500,1.534100,1.615961,0.200000
1000,1.612800,1.612190,0.200000
1500,1.611300,1.611408,0.200000
2000,1.612400,1.610585,0.200000
2500,1.610400,1.610246,0.200000
3000,1.611400,1.609869,0.200000
3500,1.609200,1.609843,0.200000
4000,1.609800,1.609734,0.200000
4500,1.609500,1.609701,0.200000
5000,1.609700,1.609645,0.200000


[I 2023-10-25 14:59:46,188] Trial 1 finished with value: 0.2 and parameters: {'learning_rate': 0.0002697965518807872, 'batch_size': 8, 'num_train_epochs': 2}. Best is trial 0 with value: 0.633.


Step,Training Loss,Validation Loss,Accuracy
500,1.609800,1.609551,0.200000
1000,1.609600,1.609532,0.200000
1500,1.609500,1.609523,0.200000
2000,1.609800,1.609482,0.200000
2500,1.609500,1.609473,0.200000
